#### Three Ways to save the models
```
torch.save(arg, PATH) #uses python pickel module to serialize the model

torch.load(PATH)

model.load_state_dict(arg)

1. complete model 
(Lazy approch)
```
torch.save(model, PATH)

#model classes must be defined somewhere
model = torch.load(PATH)

model.eval()

dis adv: serialized data bount to specific classes and exact directory structure  when model is saved

2. STATE DICT
```
torch.save(model.state_dict(), PATH)

# model must be created again with parameters
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()

In [2]:
import torch
import torch.nn as nn


In [4]:
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = nn.Sigmoid(self.linear(x))
        return y_pred

model  = Model(n_input_features=6)

Saving the model

In [8]:
FILE = 'models\\model.pth'

torch.save(model, FILE)

Loading the model

In [25]:
#1-type
loaded_model = torch.load('models\\model.pth', weights_only=False)
loaded_model.eval()


Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [22]:

for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2798,  0.1442,  0.2186, -0.0446,  0.1144,  0.3999]],
       requires_grad=True)
Parameter containing:
tensor([-0.2020], requires_grad=True)


#2-type -> packs the weights parameters with the model itself ease to load the model


In [33]:
print(model.state_dict())

OrderedDict({'linear.weight': tensor([[ 0.2798,  0.1442,  0.2186, -0.0446,  0.1144,  0.3999]]), 'linear.bias': tensor([-0.2020])})


In [ ]:
torch.save(model.state_dict(), FILE)

In [28]:
loaded_model = torch.load(FILE)
loaded_model

OrderedDict([('linear.weight',
              tensor([[ 0.2798,  0.1442,  0.2186, -0.0446,  0.1144,  0.3999]])),
             ('linear.bias', tensor([-0.2020]))])

In [30]:
loaded_model = Model(n_input_features=6)
loaded_model.load_state_dict(torch.load(FILE))
loaded_model

Model(
  (linear): Linear(in_features=6, out_features=1, bias=True)
)

In [32]:
for param in loaded_model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.2798,  0.1442,  0.2186, -0.0446,  0.1144,  0.3999]],
       requires_grad=True)
Parameter containing:
tensor([-0.2020], requires_grad=True)


### Checkpoints

* are used to save the models during training or resume for further training




In [35]:
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.001, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None, 'params': [0, 1]}]}


In [ ]:
# check points contais all the metadata, model weights, and parametric values, learning_rate, optim_state

checkpoints = {
    'epoch': 90,
    'model_state': model.state_dict(),
    'optim_state': optimizer.state_dict()

}

torch.save(checkpoints, 'models\\checkpoints.pth')

In [ ]:
loaded_checkpoints = torch.load('models\\checkpoints.pth')
epoch = loaded_checkpoints['epoch']

model = Model(n_input_features=6)
optimizer = torch.optim.SGD(model.parameters(), lr=0)


#updating the new model parameters with the previously saved checkpoints with the load_state_dict()
model.load_state_dict(checkpoints['model_state'])
optimizer.load_state_dict(checkpoints['optim_state'])

print(optimizer.state_dict)

<bound method Optimizer.state_dict of SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)>


GPU

In [ ]:
# save on GPU, load on CPU

device = torch.device('cuda')
model.to(device)
torch.save(model.state_dict(), PATH)


device = torch.device('cpu')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location=device))

In [ ]:
# Save on GPU and Load on GPU

device = torch.device('cuda')
model.save(device)
torch.save(model.state_dict(), PATH)

model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.to(device)


In [ ]:
# Save on CPU, Load on GPU

torch.save(model.state_dict(), PATH)

device = torch.device('cuda')
model = Model(*args, **kwargs)
model.load_state_dict(torch.load(PATH, map_location='cuda:0'))
model.to(device)